In [66]:
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Enumerator
import Control.Monad.Bayes.Sampler
import Control.Monad
:load Plotting.hs


# Introduction to Monad-Bayes

This serves as an interactive alternative to [the user guide](https://monad-bayes.netlify.app/probprog.html).

We'll start by defining a probabilistic model

In [68]:
model :: MonadInfer m => Double -> m Double
model observation = do
    mean <- uniformD [-1, 0, 1]
    factor (normalPdf mean 1 observation)
    return mean

The idea of monad-bayes, and probabilistic programming languages in general is to define distributions as programs. `model` corresponds to the distribution that you would express mathematically as:

$$ P(m | o) = \frac{P(m)P(o|m)}{P(o)} = \frac{1 * \mathbb{N}(o; m, 1)}{\sum_m TODO } $$
TODO 

To orient you on the relationship between the mathematical view of a distribution and the programming one, here are some notes:

- a distribution over values of type `a` has type `MonadInfer m => m a`
- a joint distribution over values of types `a` and `b` is a distribution over a tuple: `MonadInfer m => m (a, b)`
- a conditional distribution over values of type `a` conditioned on values of type `b` is a function into a distribution: `MonadInfer m => b -> m a`

Given a value of `o`, you can think of `model` as doing the following:

- first draw from the prior over possible values of `mean` (that's the line `mean <- uniformD [-1, 0, 1]`)
- then score a draw higher according to the likelihood placed on the observation by a normal with $\mu$=`mean`
- then return the `mean`


For example, if the value observed is $0.3$, then we can calculate the distribution over the mean:



In [69]:
inferredDistribution = enumerate $ model 0.3

In [70]:
plotVega inferredDistribution

To produce this plot, we performed *inference*, to obtain the exact form of the distribution represented by `model`. Because the only random variable in `model` had a support that was small and discrete (the set $\{-1, 0, 1\}$), performing this inference exactly was straightforward.

`enumerate` is the exact inference method offered by monad-bayes.

You are encouraged to change `model` in a number of ways and observe how the results change:
- try changing the prior (currently `uniformD [-1, 0, 1]`)
- try changing the score (currently `factor (normalPdf mean 1 observation)`)
- try changing the types of the observation and latent variable (i.e. `mean`)

If you are familiar with Haskell, then it should be clear that the class of distributions you can express in this way is very broad, since we have monadic control flow. For example, you could build:

In [71]:
lengthDist observations = do
    clusterAssignments <- mapM model observations
    return $ (> 4) $ length $ filter (==1) clusterAssignments

Given a set of observations, this is the distribution over whether the number of observations which belong to the mean $1$ cluster is more or less than $4$. Consider the hassle of defining this with an equation, and you'll see why probabilistic programming is appealing.

In [72]:
observations <- sampleIO $ replicateM 9 $ normal 0 1
plotVega $ enumerate $ lengthDist observations

Exact sampling is pretty limited. For models with continuous random variables, or large discrete ones, it is a no-go. 

The broader goal is to be able to define your distribution of interest, like `model`, and then apply different inference technique, usually approximate, to it. This is what monad-bayes (and other probabilistic programming languages) enable. See the following tutorials for details.

: 